<a href="https://colab.research.google.com/github/nurfnick/Metaheuristics/blob/main/Homework7/SimmulatedAnnealing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Meta-Heuristics Homework 7 Simmulated Annealing

In [1]:
import copy
from random import Random   #need this for the random number generation -- do not change
import numpy as np


In [2]:
#to setup a random number generator, we will specify a "seed" value
#need this for the random number generation -- do not change
seed = 51132023
myPRNG = Random(seed)

#to get a random number between 0 and 1, use this:             myPRNG.random()
#to get a random number between lwrBnd and upprBnd, use this:  myPRNG.uniform(lwrBnd,upprBnd)
#to get a random integer between lwrBnd and upprBnd, use this: myPRNG.randint(lwrBnd,upprBnd)

#number of elements in a solution
n = 150

#create an "instance" for the knapsack problem
value = []
for i in range(0,n):
    value.append(round(myPRNG.triangular(5,1000,200),1))

weights = []
for i in range(0,n):
    weights.append(round(myPRNG.triangular(10,200,60),1))

#define max weight for the knapsack
maxWeight = 2500


In [7]:
solutionsChecked = 0

#function to evaluate a solution x
def evaluate(x):

    a=np.array(x)
    b=np.array(value)
    c=np.array(weights)

    totalValue = np.dot(a,b)     #compute the value of the knapsack selection
    totalWeight = np.dot(a,c)    #compute the weight value of the knapsack selection

    if totalWeight > maxWeight:
         #print ("Oh no! The solution is infeasible!  What to do?  What to do?")   #you will probably want to change this...
         totalValue = totalValue - 500*(totalWeight - maxWeight)

    return [totalValue, totalWeight]   #returns a list of both total value and total weight


#here is a simple function to create a neighborhood
#1-flip neighborhood of solution x
def neighborhood(x):

    nbrhood = []

    for i in range(0,n):
        nbrhood.append(x[:])
        if nbrhood[i][i] == 1:
            nbrhood[i][i] = 0
        else:
            nbrhood[i][i] = 1

    return nbrhood
#take all the 1's and 0's and slide them around, keeping the order.
def neighborhood_slide(x):

    nbrhood = []

    for i in range(0,len(x)):
        newlist = x[i+1:]
        newlist.extend(x[:i+1])
        nbrhood.append(newlist)
    return nbrhood


def neighborhood_slide_and_flip(x):
  nbrhood = []

  for i in neighborhood(x):
    nbrhood.extend(neighborhood_slide(i))

  return nbrhood


#create the initial solution
def initial_solution_zero():
    x = []   #i recommend creating the solution as a list

    for i in range(n):
        x.append(0)

    return x

def initial_solution_random():
  x = []

  for i in range(n):
    x.append(myPRNG.randint(0,1))

  return x

def initial_solution_random_start_feasible():
  x = []

  for i in range(n):
    x.append(myPRNG.randint(0,1))

  i = myPRNG.randint(0,n-1)
  while evaluate(x)[1] > maxWeight:
    x[i] = 0
    i=myPRNG.randint(0,n-1)

  return x
#varaible to record the number of solutions evaluated
solutionsChecked = 0

x_curr = initial_solution_zero()  #x_curr will hold the current solution
x_best = x_curr[:]           #x_best will hold the best solution
f_curr = evaluate(x_curr)    #f_curr will hold the evaluation of the current soluton
f_best = f_curr[:]


In [4]:
def probability_assignment(x,T):
  if x>0:
    return 1
  else:
    return np.exp(x/T)

## Exponential Cooling

In [5]:
#begin local search overall logic ----------------
x_curr = initial_solution_random_start_feasible()  #x_curr will hold the current solution
x_best = x_curr[:]           #x_best will hold the best solution
f_curr = evaluate(x_curr)    #f_curr will hold the evaluation of the current soluton
f_best = f_curr[:]



solutionsChecked = 0
bettersolutionsfound = 0
failedToExit = 0

T = 10000

for j in range(500):

  for i in range(100):
    Neighborhood = neighborhood(x_curr)   #create a list of all neighbors in the neighborhood of x_curr
    p = 0
    howLongInWhile = 0 #need to check I don't get stuck in next while too long
    while p ==0: #getting new value allows for a break out of here
      solutionsChecked += 1
      howLongInWhile +=1
      apval = myPRNG.random() #ranomd value to compare to
      s = Neighborhood[myPRNG.randint(0,len(Neighborhood)-1)] #one random neighbor
      if evaluate(s)[0]>f_curr[0]: #better solution then current
        x_curr = s[:]
        f_curr = evaluate(s)[:]
        p =1
      elif probability_assignment(evaluate(s)[0]-f_curr[0],T)>apval : #worse solution so see if you should use it randomly
        x_curr = s[:]
        f_curr = evaluate(s)[:]
        p=1
      elif howLongInWhile >150: #a way out of this while
        failedToExit +=1
        break
  if failedToExit > 1000: #a way out of the last while
    break


  T = 0.99*T #exponential cooling

print ("\nFinal number of solutions checked: ", solutionsChecked)
print ("Best value found: ", f_curr[0])
print ("Weight is: ", f_curr[1])
print ("Total number of items selected: ", np.sum(x_curr))
print ("Best solution: ", x_curr)


Final number of solutions checked:  500448
Best value found:  23704.500000000004
Weight is:  2452.4
Total number of items selected:  42
Best solution:  [0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1]


# Cauchy Cooling

In [8]:
#begin local search overall logic ----------------
x_curr = initial_solution_random_start_feasible()  #x_curr will hold the current solution
x_best = x_curr[:]           #x_best will hold the best solution
f_curr = evaluate(x_curr)    #f_curr will hold the evaluation of the current soluton
f_best = f_curr[:]



solutionsChecked = 0
failedToExit = 0


T0 = 10000
T = T0

for j in range(1000):

  for i in range(100):
    Neighborhood = neighborhood(x_curr)   #create a list of all neighbors in the neighborhood of x_curr
    p = 0
    howLongInWhile = 0
    while p ==0:
      solutionsChecked += 1
      howLongInWhile +=1
      apval = myPRNG.random()
      s = Neighborhood[myPRNG.randint(0,len(Neighborhood)-1)]
      if evaluate(s)[0]>f_curr[0]:
        x_curr = s[:]
        f_curr = evaluate(s)[:]
        p =1
      elif probability_assignment(evaluate(s)[0]-f_curr[0],T)>apval :
        x_curr = s[:]
        f_curr = evaluate(s)[:]
        p=1
      elif howLongInWhile >150:
        failedToExit +=1
        break
  if failedToExit > 1000:
    break


  T = T0/(1+j)

print ("\nFinal number of solutions checked: ", solutionsChecked)
print ("Best value found: ", f_curr[0])
print ("Weight is: ", f_curr[1])
print ("Total number of items selected: ", np.sum(x_curr))
print ("Best solution: ", x_curr)


Final number of solutions checked:  363743
Best value found:  23288.700000000004
Weight is:  2464.4000000000005
Total number of items selected:  42
Best solution:  [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1]
